In [11]:
!pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.7 MB/s eta 0:00:00


In [9]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import pandas as pd

In [6]:
tokenizer = AutoTokenizer.from_pretrained("razent/SciFive-large-Pubmed_PMC-MedNLI")
SciFive_model = AutoModelForSeq2SeqLM.from_pretrained("razent/SciFive-large-Pubmed_PMC-MedNLI")

sent_1 = "the patient prescribed by antibiotic"
sent_2 = "The patient has an infection"
text =  f"mednli: sentence1: {sent_1} sentence2: {sent_2}"

encoding = tokenizer.encode_plus(text, padding='max_length', max_length=256, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]

outputs = SciFive_model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=8,
    early_stopping=True
)

for output in outputs:
    inference = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(inference)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


entailment


In [7]:
def find_pre_and_hyp(query):
    start_pre = query.find("[PRE]") + len("[PRE]")
    end_pre = query.find("[HYP]")
    start_hyp = query.find("[HYP]") + len("[HYP]")
    end_hyp = query.find("OUTPUT:")
    premise = query[start_pre:end_pre].strip()
    hypothesis = query[start_hyp:end_hyp].strip()

    return premise,hypothesis

In [8]:
def get_answer(premise,hypothesis):
    text =  f"mednli: sentence1: {premise} sentence2: {hypothesis}"
    encoding = tokenizer.encode_plus(text, padding='max_length', max_length=256, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
    outputs = SciFive_model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=8,
    early_stopping=True)
    for output in outputs:
        line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return line

In [12]:
test_data = pd.read_parquet('test-00000-of-00001-47685aa42db61e77.parquet', engine='fastparquet')
train_data = pd.read_parquet('train-00000-of-00001-210cfe9263b99806.parquet', engine='fastparquet')
valid_data = pd.read_parquet('valid-00000-of-00001-cc552de6d1a6fa4b.parquet', engine='fastparquet')
test_data

,id,query,answer,choices,gold
0,MedNLI0,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1,MedNLI1,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
2,MedNLI2,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
3,MedNLI3,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
4,MedNLI4,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
...,...,...,...,...,...
1417,MedNLI1417,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
1418,MedNLI1418,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
1419,MedNLI1419,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1420,MedNLI1420,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1


In [13]:
count = 0
right = 0
for query,answer in zip(test_data['query'],test_data['answer']):
    count = count + 1
    premise,hypothesis = find_pre_and_hyp(query)
    an = get_answer(premise,hypothesis)
    if an == answer:
       right = right + 1
print(right/count)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


0.8649789029535865
